In [1]:
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
#from utils.polyhierarchy_comparison_measures import SP_weighted_embeddings_wordnet,SP_max_embeddings_wordnet
import itertools
import logging
import operator
import numpy as np
from scipy import spatial
import networkx as nx
import glob
from scipy import stats
from numpy.random import randn
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
wn_lemma = WordNetLemmatizer()

In [2]:
def set_diag(self, values): 
    n = min(len(self.index), len(self.columns))
    self.values[tuple([np.arange(n)] * 2)] = values




In [3]:
import numpy as np
import matplotlib.pyplot as plt; plt.ion()
import networkx as nx

def find_lowest_common_ancestor(graph, a, b):
    """
    Find the lowest common ancestor in the directed, acyclic graph of node a and b.
    The LCA is defined as on

    @reference:
    https://en.wikipedia.org/wiki/Lowest_common_ancestor

    Notes:
    ------
    This definition is the opposite of the term as it is used e.g. in biology!

    Arguments:
    ----------
        graph: networkx.DiGraph instance
            directed, acyclic, graph

        a, b:
            node IDs

    Returns:
    --------
        lca: [node 1, ..., node n]
            list of lowest common ancestor nodes (can be more than one)
    """

    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    # get ancestors of both (intersection)
    common_ancestors = list(nx.ancestors(graph, a) & nx.ancestors(graph, b))
    
    # get sum of path lengths
    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = nx.shortest_path_length(graph, c, a) \
                                  + nx.shortest_path_length(graph,  c,b)

    # print common_ancestors
    # print sum_of_path_lengths
    
    # return minima
    minima, = np.where(sum_of_path_lengths == np.min(sum_of_path_lengths))
    #print(minima)
    return min(sum_of_path_lengths)
    #return [common_ancestors[ii] for ii in minima]

In [4]:
graph_file = './Graph/Noungraph_wordsimilarity.csv'
df = pd.read_csv(graph_file)
G = nx.from_pandas_edgelist(df,
                            source='parent',
                            target='child',
                            create_using=nx.DiGraph())
# find level of node(shortest path from root to current node)
optional_attrs = nx.shortest_path_length(G ,'root')
nx.set_node_attributes(G ,  optional_attrs, 'node_level' )

In [5]:
# create dictionary for the input words and 3 related synsets in the wordnet
wn_lemma = WordNetLemmatizer()
def word_synset3_dict(concept_names, word_type='noun'):
    leaf_nodes = dict()
    for c in concept_names:
        
        ls= []
        
        word1 = wn_lemma.lemmatize(c)
        
        if word_type=='verb':
            count = 0
            for result in wn.synsets(word1, pos = wn.VERB): # retrieve all concepts related to this word
                if count<20:
                    ls.append(result.name())
                    count+=1
                if ls:
                    leaf_nodes[word1] = ls
        if word_type=='noun':
            count = 0
            for result in wn.synsets(word1, pos = wn.NOUN): # retrieve all concepts related to this word

                # only add corresponding noun synsets '.n.' in result.name() and 
                if count<20:
                    ls.append(result.name())
                    count+=1
                if ls:
                    leaf_nodes[word1] = ls

        
    return leaf_nodes



In [6]:
# create dictionary for the input words and 3 related synsets in the wordnet

def word_synsetall_dict(concept_names, pos):
    leaf_nodes = dict()
    for c in concept_names:
        ls= []
        count = 0
        word1 = wn_lemma.lemmatize(c)
        for result in wn.synsets(word1): # retrieve all concepts related to this word
                ls.append(result.name())
        if ls:
            leaf_nodes[word1] = ls
    return leaf_nodes




In [7]:
# CREATE embeddings based on common ancestors between two nodes.
def SP_embeddings_wordnet(filename, target_filename,rootnode, word_pairs, lambda_factor=0.7, concept_dict=None):
    df = pd.read_csv(filename)
    # Create the Directed Graph 
    try:
        G = nx.from_pandas_edgelist(df,
                            source='parent',
                            target='child',
                            create_using=nx.DiGraph())
        # find level of node(shortest path from root to current node)
        optional_attrs = nx.shortest_path_length(G ,rootnode)
        nx.set_node_attributes(G ,  optional_attrs, 'node_level' )
        
        #word_pairs = list(itertools.product(list(concept_dict.keys()), repeat=2)) # create pair of all nodes 
       
        ancestors_distance = {}
        for word in word_pairs:        
            if word[0]== word[1]:
                ancestors_distance[word]=1
            else:
                common_ancestor = {} 
                # fetch synsets from the graph for both words
                if concept_dict:
                    concepts_word1 = concept_dict[word[0]]
                    concepts_word2 = concept_dict[word[1]]
                else:
                    print (word)
                    break
                pairs = list(itertools.product(concepts_word1, concepts_word2)) # create pair of all nodes 
                
                ls_similarity = []
                
                distance_list=[]
                distance_dict= {}
                for i in pairs:
                    minima = find_lowest_common_ancestor (G, i[0], i[1])
                    sim_m = lambda_factor**(( minima)/2) # divide may lead to float lamads , 2, 2,5                 
                    #distance_dict[i] = np.max(ls_distance)
                    distance_list.append (sim_m) # choose the distance for lowest common ancestor for given concept pair

                
                ancestors_distance[word] =   np.max(distance_list)


                
        chunked_data = [[k[0],k[1], v] for k, v in ancestors_distance.items()]
        df_nodes = pd.DataFrame(chunked_data)
        df_nodes = df_nodes.rename(columns= {0:'node1', 1:'node2', 2:'weight'})
        
        
        # create adjancey matrix
        df_nodes.to_csv(target_filename)
        
        
    except BaseException:
        logging.exception("An exception was thrown!")

In [39]:
data_files = glob.glob("datasets/*.csv")
data_files = ['datasets/NOUN_rw.csv']

In [71]:
graph_file = './Graph/graph_wordsimilarity.csv'
#graph_file = './Graph/Noungraph_wordsimilarity.csv'
#data_files = glob.glob("datasets/*.csv")
#data_files =["datasets/NOUN_mc-30.csv"]
lambda_factor= 0.55#0.755 #0.9
strategy = 'SP'
column_names = ['word1', 'word2', 'sim']
root_node = 'root'
count =0
for file in data_files:
    print (file)
    if count==10:
        break
    else:
        count +=1
        targetfile = file.split('/')[1]
        word_list = []
        df = pd.read_csv(file) #,  names = column_names)#sep=';',
        
        word_list.append(df['word1'])
        word_list.append(df['word2'])  
        word_list = list(set([item for sublist in word_list for item in sublist]))
        
        
        # create dictionary of word synsets from given word list
        if file.split('/')[1].split('_')[0].lower() =='noun':
            print (file.split('/')[1])
            leaf_nodes = word_synset3_dict(word_list, 'noun')

            wd1 = [wn_lemma.lemmatize(w) for w in df['word1']]
            wd2 = [wn_lemma.lemmatize(w) for w in df['word2']]
            word_pairs = list(zip(wd1, wd2))
            
            ls = leaf_nodes.keys() 
            # call all functions for each dataset
            
            if leaf_nodes :
                
                SP_embeddings_wordnet(graph_file,  './sim_scores/SP/'+targetfile, root_node ,word_pairs,lambda_factor, leaf_nodes)
            else:
                print ('No data found')


        elif file.split('/')[1].split('_')[0].lower() =='verb':
            print (file.split('/')[1])
            leaf_nodes = word_synset3_dict(word_list, 'verb')
            #
            wd1 = [wn_lemma.lemmatize(w) for w in df['word1']]
            wd2 = [wn_lemma.lemmatize(w) for w in df['word2']]
            word_pairs =list(zip(wd1, wd2))
        
            ls = leaf_nodes.keys() 
           
            # call all functions for each dataset
            if leaf_nodes :
                
                SP_embeddings_wordnet(graph_file,  './sim_scores/SP/'+targetfile, root_node ,word_pairs,lambda_factor, leaf_nodes)
            else:
                pass
                #print ('No data found')
        else:
            #leaf_nodes = None
            pass
        
    
        

datasets/NOUN_rw.csv
NOUN_rw.csv


In [72]:
ls = []
for i in wd2:
    if i in leaf_nodes:
        pass
    else:
        ls.append(i)

In [73]:
ls

[]

In [74]:
ls1 = []
for i in wd1:
    if i in leaf_nodes:
        pass
    else:
        ls1.append(i)

In [75]:
ls1

[]

In [51]:
df = pd.read_csv('datasets/NOUN_rw.csv')
df = df.loc[~df['word2'].isin(ls)]
df = df.loc[~df['word1'].isin(ls1)]


#df.to_csv()

In [52]:
df.loc[df['word2'].isin(ls)]

,word1,word2,similarity


In [76]:
pearson_ls = []
spearman_ls = [] 
result_dict = {}
goldDatapath= './datasets/'
sim_data = glob.glob("./sim_scores/SP/*.csv")
column_names = ['word1', 'word2', 'sim']
algos = [  'lch']
columns =['SimLex222','Agirre201', 'Gerz', 'Millers', 'Rel122', 'wordsim353', 'yang' ]
df_score =pd.DataFrame(columns=columns)
index_ls = []
for file in sim_data:
    print (file)
    algoname = file.split('/')[2].split('sss')[0]
    
    filename = file.split('/')[-1]
    print ()
    for algoname in algos:
        # read similarity file
        df_r = pd.read_csv(file)
        # read human similarity score
        df = pd.read_csv(goldDatapath+filename)
        score_human = df['similarity'].values
        norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
        norm1 = np.round(norm1,2)

        algo_score = np.round( df_r['weight'].values,2)
        # normalize score
        #if (df_r['weight'] > 1).any():
        #algo_score= [(float(i)-min(algo_score))/(max(algo_score )-min(algo_score )) for i in algo_score ] # normalize score in 0-1
        print (len(algo_score))
        print (len(norm1))
        # calculate Pearson's correlation
        corrp, _ = pearsonr(norm1, algo_score)
        pearson_ls.append(corrp) 
        print (np.round(corrp,2))
        corrs, _ = spearmanr(norm1, algo_score)
        #print('Spearmans correlation: %.3f' % corr)
        spearman_ls.append(corrs) 
        print (np.round(corrs,2))
#df_score.loc[len(df_score.index)+1] =pearson_ls
    

./sim_scores/SP/NOUN_rg-65.csv

65
65
0.81
0.76
./sim_scores/SP/NOUN_mturk-287.csv

243
243
0.33
0.26
./sim_scores/SP/NOUN_wordsim353-rel.csv

248
248
0.07
0.0
./sim_scores/SP/NOUN_mc-30.csv

30
30
0.72
0.71
./sim_scores/SP/NOUN_wordsim353-sim.csv

201
201
0.61
0.59
./sim_scores/SP/NOUN_rw.csv

865
910


ValueError: x and y must have the same length.

In [77]:
df_r = pd.read_csv('./sim_scores/SP/NOUN_rw.csv')
# read human similarity score
df = pd.read_csv('datasets/NOUN_rw.csv')

algo_score = np.round( df_r['weight'].values,2)
#df = df.loc[df.word2.isin(df_r.node2.values)]

#ls = list(set(df['word1'].values) - set(df_r['node1'].values))
#print (len(ls))
#df = df.loc[~df.word1.isin(ls)]


score_human = df['similarity'].values
norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
norm1 = np.round(norm1,2)

print (len(algo_score))
print (len(norm1))


# calculate Pearson's correlation
corrp, _ = pearsonr(norm1, algo_score)
pearson_ls.append(corrp) 
print (np.round(corrp,2))
corrs, _ = spearmanr(norm1, algo_score)
#print('Spearmans correlation: %.3f' % corr)
spearman_ls.append(corrs) 
print (np.round(corrs,2))
#df_score.lo

865
910


ValueError: x and y must have the same length.

### Check results 
   
  

In [50]:
df = pd.DataFrame(result_dict).T
df.columns = columns
df.style.highlight_max(color = 'lightgreen', axis = 0)

,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
SP : spearman,0.509738,0.283972,0.196846,0.657737,0.473691,0.366731,0.198774,0.496549
SP : Pearson,0.489521,0.254977,0.133855,0.655934,0.467584,0.379607,0.189695,0.487518
Leacock : spearman,-0.383200,-0.114225,0.035563,-0.266795,-0.023329,-0.262912,-0.214483,-0.131478
Leacock : Pearson,-0.300321,-0.101877,-0.006449,-0.340009,0.017026,-0.263540,-0.225336,-0.128977


In [53]:
df_sematch = pd.read_csv('sematch_KRbased_result.csv')
df_sematch = df_sematch.set_index('Unnamed: 0')
df_final  = round( pd.concat([df, df_sematch]),2)
df_final.style.highlight_max(color = 'lightgreen', axis = 0)


,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
SP : spearman,0.510000,0.280000,0.200000,0.660000,0.470000,0.370000,0.200000,0.500000
SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000


In [5]:
pd.read_csv('sem_Evaluation_KR.csv').style.highlight_max(color = 'lightgreen', axis = 0)

,Unnamed: 0,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
0,SP : spearman,0.510000,0.280000,0.200000,0.680000,0.470000,0.370000,0.200000,0.500000
1,SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
2,Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
3,Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
4,wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
5,wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
6,path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
7,path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
8,lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
9,lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000


In [7]:
pd.read_csv('sem_Evaluation_IC.csv').style.highlight_max(color = 'lightgreen', axis = 0)

,Unnamed: 0,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
0,SP : spearman,0.510000,0.280000,0.200000,0.680000,0.470000,0.370000,0.200000,0.500000
1,SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
2,Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
3,Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
4,wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
5,wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
6,path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
7,path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
8,lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
9,lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000
